<center>
<h1><b>Word Embedding Based Answer Evaluation System for Online Assessments (WebAES)</b></h1>
<h3>A smart system to automate the process of answer evaluation in online assessments.</h3>
<h5> LDA + BERT Model for WebAES</h5>

In [1]:
# To perform text pre-processing
import string

# Natural Language Toolkit
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

# Set of stopwords in English
en_stopwords = set(stopwords.words('english'))

# To load text corpus and pre-trained LDA model
from gensim import corpora, models
import gensim.downloader as api

# To perform sentence encoding using BERT model
from sentence_transformers import SentenceTransformer

# To determine similarity between 2 vectors
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\prath\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# Load pre-trained and saved LDA model
lda_model = models.LdaModel.load('WebAES_LDA_Model.model')


# Load text8 corpus and convert to list of documents
text8_corpus = api.load('text8')
text8_corpus = [doc for doc in text8_corpus]

# List containing list of tokens from each document of text8 corpus
list_of_list_of_tokens = []

# For each document in the text8 corpus
for i in range(len(text8_corpus)):
    # Remove stopwords from each document
    text8_corpus[i] = [w for w in text8_corpus[i] if w not in en_stopwords]
    
    # Add list of tokens for document to list of list of tokens
    list_of_list_of_tokens.append(text8_corpus[i])
    
dictionary_LDA = corpora.Dictionary(list_of_list_of_tokens)

In [3]:
# Function to perform text pre-processing operations
def preprocess(text):
    # Remove punctuations
    text = text.translate(str.maketrans('', '', string.punctuation)).lower()
    
    # Remove stopwords
    text = ' '.join([w for w in text.split() if not w.lower() in en_stopwords])
    
    # Split text into list of tokens
    text_tokens = text.split()
    
    # Return list of tokens
    return text_tokens

In [4]:
# Function to extract topic from text
def get_topic_prob(text_tokens):
    max_prob_topic, max_prob = 0, 0
    
    # Get topic probabilities for given text using LDA model
    topic_probs = lda_model[dictionary_LDA.doc2bow(text_tokens)]
    
    # Select topic with highest probability
    for topic in topic_probs:
        topic_index, topic_prob = topic[0], topic[1]
        if topic_prob > max_prob:
            max_prob = topic_prob
            max_prob_topic = topic_index
            
    # Return topic with max probability and probabilty of that topic
    return max_prob_topic, max_prob

In [5]:
# Get document embeddings for list of documents
def get_bert_embeddings(docs):
    # Load pre-trained BERT model
    BERT_model = SentenceTransformer('bert-base-nli-mean-tokens')
    
    # Encode documents using BERT model
    doc_embeddings = BERT_model.encode(docs)
    
    # Return document embeddings
    return doc_embeddings

In [6]:
# Determine similarity between documents
def similarity(doc_embeddings):
    # Similarity score based on cosine similarity measure
    sim_score = cosine_similarity([doc_embeddings[0]], doc_embeddings[1:])[0][0]
    
    # Return similarity score
    return sim_score

In [7]:
# Function to automatically evaluate test
def evaluate(expected, response):
    # Pre-rpocessing for expected answer
    exp_ans_tokens = preprocess(expected)
    
    # Get topic for expected answer
    exp_ans_topic, exp_ans_topic_prob = get_topic_prob(exp_ans_tokens)
    
    # Pre-processing for student's response
    stu_ans_tokens = preprocess(response)
    
    # Get topic for student's answer
    stu_ans_topic, stu_ans_topic_prob = get_topic_prob(stu_ans_tokens)
    
    # List of documents
    docs = [expected, response]
    
    # Get document embeddings for expected answer and student response
    doc_embeddings = get_bert_embeddings(docs)
    
    # Get similarity score based on documents embeddings
    sim_score = similarity(doc_embeddings)
    
    # Calculate marks iff topics match for expected answer and student's response
    if stu_ans_topic==exp_ans_topic:
        marks = (stu_ans_topic_prob/exp_ans_topic_prob)*sim_score*10
    # If topics do not match, marks are 0
    else:
        marks = 0
        
    # Return marks scored
    return marks

In [8]:
# Function to start test
def take_test(question, max_marks, expected_answer):
    # Get student's response
    student_response = input('\n' + question + ' ({} marks)\n\n'.format(max_marks))
    
    # Determine score for student's response
    score = round(evaluate(expected_answer, student_response), 2)
    
    # Display marks scored
    print('\nYou have scored {} marks out of {}.'.format(score, max_marks))

In [9]:
# Sample question
question = 'Give PEAS description for medical diagnosis system.'

# Maximum marks for question
max_marks = 10

# Answer expected by faculty
expected_answer = '''The performance measure for medical diagnosis system may include the number of patients healed by 
correctly and accurately diagnosing diseases. For example, the performance measure may be the percentage of cases diagnosed 
correctly by the system. The environment for a medical diagnosis system includes patients and their vital signs. This 
environment is fully observable, dynamic and complete. The actuators include display screens and alert systems that send 
feedback to doctors. Sensors include equipment including medical sensors as well as medical images.'''

# Start test and display result
take_test(question, max_marks, expected_answer)


Give PEAS description for medical diagnosis system. (10 marks)

The percentage of cases correctly identified by the medical diagnosis system is the performance measure for a medical diagnosis  system. The performance measure can be described as the number of cases out of every one hundred cases that were diagnosed correctly  by the system. In this case, the environment is complete, observable and static. The feedback system which gives the result of diagnosis  to a doctor is the actuator. Sensors include medical sensors such as ECG equipment as well as cameras to monitor images of X Rays.

You have scored 9.3 marks out of 10.
